In [1]:
# install package
using Pkg
Pkg.instantiate()
import Pkg
Pkg.add("TimeZones")
using PowerModels
Pkg.activate(".")
Pkg.status()

Pkg.add("Xpress")
Pkg.add("JSON")
Pkg.add("JuMP")
Pkg.add("Ipopt")
Pkg.add("InfrastructureModels")
Pkg.add("PowerModels")
Pkg.add("Memento")

using Xpress

time_start = time()
println("start pre-compilation")

using Distributed
using SparseArrays


using JSON

using JuMP

using Ipopt

using InfrastructureModels
using PowerModels
using Memento

const LOGGER = getlogger("GOC")

import Statistics: mean

println("pre-compilation finished ($(time() - time_start))")

   Updating registry at `C:\Users\Rita\.julia\registries\General`


   Updating git-repo `https://github.com/JuliaRegistries/General.git`


  Resolving package versions...
   Updating `C:\Users\Rita\Desktop\Project - code-RUODAN WANG\Project.toml`
 [no changes]
   Updating `C:\Users\Rita\Desktop\Project - code-RUODAN WANG\Manifest.toml`
 [no changes]
 Activating environment at `C:\Users\Rita\Desktop\Project - code-RUODAN WANG\Project.toml`


Status `C:\Users\Rita\Desktop\Project - code-RUODAN WANG\Project.toml`
  [2030c09a] InfrastructureModels v0.5.3
  [b6b21f68] Ipopt v0.6.3
  [682c06a0] JSON v0.21.0
  [4076af6c] JuMP v0.21.3
  [f28f55f0] Memento v1.1.1
  [c36e90e8] PowerModels v0.17.2
  [f269a46b] TimeZones v1.3.2
  [9e70acf3] Xpress v0.11.0


  Resolving package versions...
   Updating `C:\Users\Rita\Desktop\Project - code-RUODAN WANG\Project.toml`
 [no changes]
   Updating `C:\Users\Rita\Desktop\Project - code-RUODAN WANG\Manifest.toml`
 [no changes]
  Resolving package versions...
   Updating `C:\Users\Rita\Desktop\Project - code-RUODAN WANG\Project.toml`
 [no changes]
   Updating `C:\Users\Rita\Desktop\Project - code-RUODAN WANG\Manifest.toml`
 [no changes]
  Resolving package versions...
   Updating `C:\Users\Rita\Desktop\Project - code-RUODAN WANG\Project.toml`
 [no changes]
   Updating `C:\Users\Rita\Desktop\Project - code-RUODAN WANG\Manifest.toml`
 [no changes]
  Resolving package versions...
   Updating `C:\Users\Rita\Desktop\Project - code-RUODAN WANG\Project.toml`
 [no changes]
   Updating `C:\Users\Rita\Desktop\Project - code-RUODAN WANG\Manifest.toml`
 [no changes]
  Resolving package versions...
   Updating `C:\Users\Rita\Desktop\Project - code-RUODAN WANG\Project.toml`
 [no changes]
   Updating `C:\Users\Rita

start pre-compilation
pre-compilation finished (1.1740000247955322)


In [2]:
# this function is to compute the basic ACOPF problem 
# the input is the dataset of the whole network 
# the output is solution of generation value and voltage level value
function run_ac_opf1(network)
    bus_number = length(network["bus"])
    gen_number = length(network["gen"])
    branch_number = length(network["branch"])

    model_ac = Model(with_optimizer(Ipopt.Optimizer))
    @variable(model_ac, flow_p[1:branch_number])
    @variable(model_ac, flow_q[1:branch_number])
    @variable(model_ac,voltage_phase[1:bus_number]) 
    @variable(model_ac,voltage_level[1:bus_number]) 
# @variable(model_dc, gen_p[1:gen_number])
    @variable(model_ac, bus_p[1:bus_number])
    @variable(model_ac, bus_q[1:bus_number])
    @variable(model_ac, gen_p[1:gen_number])
    @variable(model_ac, gen_q[1:gen_number])
    @variable(model_ac, gen_cost[1:gen_number])
    #@variable(model_ac, bus_add_cut1[1:bus_number])
    #@variable(model_ac, bus_add_cut2[1:bus_number])

    beta_g = []
    for i in 1:gen_number
        gen_index = string(i)
        push!(beta_g,network["gen"][gen_index]["gen_bus"])
    end
    
   bus_gen = zeros(1,bus_number)
   for i in 1:gen_number
        index = beta_g[i]
        bus_gen[index] = 1
   end
         
   bus_pd = zeros(1,bus_number)
    for i in 1:length(network["load"])
        load_index = string(i)
        bus_name = network["load"][load_index]["load_bus"]
        bus_demand = network["load"][load_index]["pd"]
        bus_pd[bus_name] = bus_demand
    end

    bus_qd = zeros(1,bus_number)
    for i in 1:length(network["load"])
        load_index = string(i)
        bus_name = network["load"][load_index]["load_bus"]
        bus_demand = network["load"][load_index]["qd"]
        bus_qd[bus_name] = bus_demand
    end

    ab_l = []
    for i in 1:branch_number
        abl = []
        branch_index = string(i)
        push!(abl,network["branch"][branch_index]["f_bus"])
        push!(abl,network["branch"][branch_index]["t_bus"])
        push!(ab_l,abl)
    end

    A_bl=zeros(bus_number,branch_number)
    from_bus = 0
    to_bus = 0
    for i in 1:branch_number
        from_bus = ab_l[i][1]
        to_bus = ab_l[i][2]
        A_bl[(i-1)*bus_number+from_bus] = -1
        A_bl[(i-1)*bus_number+to_bus] = 1
    end 

    bus_p_max = zeros(1,bus_number)
        for i in 1:gen_number
            gen_index = string(i)
            bus_name = network["gen"][gen_index]["gen_bus"]
            bus_produce_max = network["gen"][gen_index]["pmax"]
            bus_p_max[bus_name] = bus_produce_max
        end

    bus_q_max = zeros(1,bus_number)
        for i in 1:gen_number
            gen_index = string(i)
            bus_name = network["gen"][gen_index]["gen_bus"]
            bus_q_max_value = network["gen"][gen_index]["qmax"]
            bus_q_max[bus_name] = bus_q_max_value
        end

    bus_p_min = zeros(1,bus_number)
        for i in 1:gen_number
            gen_index = string(i)
            bus_name = network["gen"][gen_index]["gen_bus"]
            bus_produce_min = network["gen"][gen_index]["pmin"]
            bus_p_min[bus_name] = bus_produce_min
        end

    bus_q_min = zeros(1,bus_number)
        for i in 1:gen_number
            gen_index = string(i)
            bus_name = network["gen"][gen_index]["gen_bus"]
            bus_q_min_value = network["gen"][gen_index]["qmin"]
            bus_q_min[bus_name] = bus_q_min_value
        end

    flow_bus_p_matrix = A_bl*flow_p
    flow_bus_q_matrix = A_bl*flow_q

    bus_v_min = []
    bus_v_max = []
    for i in 1:bus_number
        bus_index = string(i)
        push!(bus_v_min,network["bus"][bus_index]["vmin"])
        push!(bus_v_max,network["bus"][bus_index]["vmax"])
    end

    F_max = []
    for i in 1:branch_number
        branch_index = string(i)
        push!(F_max,network["branch"][branch_index]["rate_a"])
    end  

    B_b = zeros(1,bus_number)
    for i in 1:length(network["shunt"])
        i_string = string(i)
        bus_index = network["shunt"][i_string]["shunt_bus"]
        bus_susceptance =  network["shunt"][i_string]["bs"]
        B_b[bus_index] = bus_susceptance
    end

    bl = []
    gl = []
    for i in 1:branch_number
        branch_index = string(i)
        reactance = network["branch"][branch_index]["br_x"]
        resistance = network["branch"][branch_index]["br_r"]
        push!(bl,-reactance/(reactance^2+resistance^2))
        push!(gl,resistance/(reactance^2+resistance^2))
    end

# gen cost
   for i in 1:gen_number
       gen_index = string(i)
       bus_index = network["gen"][gen_index]["gen_bus"]
       @constraint(model_ac,gen_p[i] == bus_p[bus_index])
       @constraint(model_ac,gen_q[i] == bus_q[bus_index])
   end

   gen_cost_con=[]
   for i in 1:gen_number
       gen_index = string(i)
       push!(gen_cost_con,@constraint(model_ac,gen_cost[i] == network["gen"][gen_index]["cost"][1]*gen_p[i]^2+network["gen"][gen_index]["cost"][2]*gen_p[i]+network["gen"][gen_index]["cost"][3]))
   end
    
    
# Generation level constraints    
   for i in 1:bus_number
        if i in beta_g
            @constraint(model_ac, bus_q[i] <= bus_q_max[i])
            @constraint(model_ac, bus_q[i] >= bus_q_min[i])
        else
            @constraint(model_ac, bus_q[i] == 0)
        end
    end

    for i in 1:bus_number
        if i in beta_g
            @constraint(model_ac, bus_p[i] <= bus_p_max[i])
            @constraint(model_ac, bus_p[i] >= bus_p_min[i])
        else
            @constraint(model_ac, bus_p[i] == 0)
        end
    end

# Kirchhoff current law
    con_p= []
    for i in 1:bus_number
        bus_index = string(i)
        push!(con_p,@constraint(model_ac, flow_bus_p_matrix[i]- bus_pd[i]== -bus_p[i] ))
    end
    con_q= []
    for i in 1:bus_number
        bus_index = string(i)
        push!(con_q,@constraint(model_ac, flow_bus_q_matrix[i]- bus_qd[i] + B_b[i]*(voltage_level[i])^2== -bus_q[i] ))
    end

# voltage constraint
    for i in 1:bus_number
        @constraint(model_ac, voltage_level[i] <= bus_v_max[i])
        @constraint(model_ac, voltage_level[i] >= bus_v_min[i])
    end

# Setting the Reference Bus (Bus 1) angles to 0
    @constraint(model_ac, voltage_phase[1] == 0)

# Thermal limits
    flow_con = []
    for i in 1:branch_number
        push!(flow_con,@constraint(model_ac, flow_p[i]^2 + flow_q[i]^2 <= F_max[i]^2))
    end 

# Kirchhoff voltage law
    p_kvl = []
    q_kvl = []
    for i in 1:branch_number
        branch_index = string(i)
        bus_fr_index = network["branch"][branch_index]["f_bus"]
        bus_to_index = network["branch"][branch_index]["t_bus"]    
        push!(p_kvl,@NLconstraint(model_ac,flow_p[i]== gl[i]*voltage_level[bus_fr_index]^2-voltage_level[bus_fr_index]*voltage_level[bus_to_index]*(gl[i]*cos(voltage_phase[bus_fr_index]-voltage_phase[bus_to_index])+bl[i]*sin(voltage_phase[bus_fr_index]-voltage_phase[bus_to_index]))))
        push!(q_kvl,@NLconstraint(model_ac,flow_q[i]== -bl[i]*voltage_level[bus_fr_index]^2-voltage_level[bus_fr_index]*voltage_level[bus_to_index]*(gl[i]*sin(voltage_phase[bus_fr_index]-voltage_phase[bus_to_index])-bl[i]*cos(voltage_phase[bus_fr_index]-voltage_phase[bus_to_index]))))
    end 

# objective
    model = @objective(model_ac, Min,sum(gen_cost))
    optimize!(model_ac)
    
    return(value.(gen_p),value.(voltage_level))
end

run_ac_opf1 (generic function with 1 method)

In [3]:
# dataset
case = "case30.m"
# transform data
case_data= parse_matpower(case)
# using IPOPT (interior point method) as ptimial probelm solver
solver = JuMP.with_optimizer(Ipopt.Optimizer, tol=1e-9, print_level=0)
# solve basic ACOPF model
result_base = run_ac_opf1(case_data)

[info | PowerModels]: extending matpower format with data: areas 3x3


┌ Warning: `with_optimizer` is deprecated. Adapt the following example to update your code:
│ `with_optimizer(Ipopt.Optimizer, max_cpu_time=60.0)` becomes `optimizer_with_attributes(Ipopt.Optimizer, "max_cpu_time" => 60.0)`.
│   caller = ip:0x0
└ @ Core :-1
┌ Warning: `with_optimizer` is deprecated. Adapt the following example to update your code:
│ `with_optimizer(Ipopt.Optimizer)` becomes `Ipopt.Optimizer`.
│   caller = run_ac_opf1(::Dict{String,Any}) at In[2]:9
└ @ Main .\In[2]:9



******************************************************************************
This program contains Ipopt, a library for large-scale nonlinear optimization.
 Ipopt is released as open source code under the Eclipse Public License (EPL).
         For more information visit http://projects.coin-or.org/Ipopt
******************************************************************************

This is Ipopt version 3.13.2, running with linear solver mumps.
NOTE: Other linear solvers might be more efficient (see Ipopt documentation).

Number of nonzeros in equality constraint Jacobian...:      727
Number of nonzeros in inequality constraint Jacobian.:      166
Number of nonzeros in Lagrangian Hessian.............:      910

Total number of variables............................:      220
                     variables with only lower bounds:        0
                variables with lower and upper bounds:        0
                     variables with only upper bounds:        0
Total number of equa

([0.42351201076708855, 0.5554793783571755, 0.21806861271949055, 0.40366592031384435, 0.14820917146513823, 0.1430649063772628], [0.9574343520219301, 0.9603049473832336, 0.9644983865260086, 0.9666421194685372, 0.9559398822185319, 0.9626007961982769, 0.94999999005078, 0.9511821762232525, 0.9925326125752869, 1.0077036194384246  …  1.0175688710919593, 1.0243395302965876, 1.0600911684963772, 1.0327690043154847, 1.0499897345110734, 1.0333574465648234, 1.0677535899936672, 0.9728150527506402, 1.0500000104541447, 1.0396282808845396])

In [4]:
#PowerModels.print_summary(result_base)
PowerModels.print_summary(case_data)

Metadata
  baseMVA: 100
  name: case30
  per_unit: true
  source_type: matpower
  source_version: 2

Table Counts
  bus: 30
  load: 20
  shunt: 2
  gen: 6
  storage: 0
  switch: 0
  branch: 41
  dcline: 0
  areas: 3


Table: bus
         bus_i, bus_type,  vmax, area
      1:     1,        3, 1.050,    1
      2:     2,        2, 1.100,    1
      3:     3,        1, 1.050,    1
      4:     4,        1, 1.050,    1
      5:     5,        1, 1.050,    1
      6:     6,        1, 1.050,    1
      7:     7,        1, 1.050,    1
      8:     8,        1, 1.050,    1
      9:     9,        1, 1.050,    1
     10:    10,        1, 1.050,    3
     11:    11,        1, 1.050,    1
     12:    12,        1, 1.050,    2
     13:    13,        2, 1.100,    2
     14:    14,        1, 1.050,    2
     15:    15,        1, 1.050,    2
     16:    16,        1, 1.050,    2
     17:    17,        1, 1.050,    2
     18:    18,        1, 1.050,    2
     19:    19,        1, 1.050,    2
     20:   

In [5]:
basic_gen_p = collect(result_base)[1]

6-element Array{Float64,1}:
 0.42351201076708855
 0.5554793783571755
 0.21806861271949055
 0.40366592031384435
 0.14820917146513823
 0.1430649063772628

In [6]:
basic_voltage_level = collect(result_base)[2]

30-element Array{Float64,1}:
 0.9574343520219301
 0.9603049473832336
 0.9644983865260086
 0.9666421194685372
 0.9559398822185319
 0.9626007961982769
 0.94999999005078
 0.9511821762232525
 0.9925326125752869
 1.0077036194384246
 0.9925326125752869
 1.039637218672915
 1.100000009602725
 ⋮
 1.0036287158262724
 1.0038376724640563
 1.0175688710919593
 1.0243395302965876
 1.0600911684963772
 1.0327690043154847
 1.0499897345110734
 1.0333574465648234
 1.0677535899936672
 0.9728150527506402
 1.0500000104541447
 1.0396282808845396

In [7]:
network_base_con = deepcopy(case_data)
for i in 1:length(network_base_con["gen"])
    network_base_con["gen"][string(i)]["pg"] = basic_gen_p[i]  
    # network_base_con["gen"][string(i)]["qg"] = basic_gen_q[i]  
end

for i in 1:length(network_base_con["bus"])
    network_base_con["bus"][string(i)]["vm"] = basic_voltage_level[i]
end

In [8]:
# this function is to compute the scopf problem with one specific branch contingency
# the input is the dataset of the whole network and the contingency
# the output is solution of the problem if it has feasible, if it is not feasible the model will return "infeasible"
function run_ac_opf_withcontingency(network,contingency)
    bus_number = length(network["bus"])
    gen_number = length(network["gen"])
    branch_number = length(network["branch"])

    model_ac = Model(with_optimizer(Ipopt.Optimizer,print_level=0))
    @variable(model_ac, flow_p[1:branch_number])
    @variable(model_ac, flow_q[1:branch_number])
    @variable(model_ac,voltage_phase[1:bus_number]) 
    @variable(model_ac,voltage_level[1:bus_number]) 
    @variable(model_ac, bus_p[1:bus_number])
    @variable(model_ac, bus_q[1:bus_number])
    @variable(model_ac, gen_p[1:gen_number])
    @variable(model_ac, gen_q[1:gen_number])
    @variable(model_ac, gen_cost[1:gen_number])

    beta_g = []
    for i in 1:gen_number
        gen_index = string(i)
        push!(beta_g,network["gen"][gen_index]["gen_bus"])
    end
    
   bus_gen = zeros(1,bus_number)
   for i in 1:gen_number
        index = beta_g[i]
        bus_gen[index] = 1
   end
         
   bus_pd = zeros(1,bus_number)
    for i in 1:length(network["load"])
        load_index = string(i)
        bus_name = network["load"][load_index]["load_bus"]
        bus_demand = network["load"][load_index]["pd"]
        bus_pd[bus_name] = bus_demand
    end

    bus_qd = zeros(1,bus_number)
    for i in 1:length(network["load"])
        load_index = string(i)
        bus_name = network["load"][load_index]["load_bus"]
        bus_demand = network["load"][load_index]["qd"]
        bus_qd[bus_name] = bus_demand
    end

    ab_l = []
    for i in 1:branch_number
        abl = []
        branch_index = string(i)
        push!(abl,network["branch"][branch_index]["f_bus"])
        push!(abl,network["branch"][branch_index]["t_bus"])
        push!(ab_l,abl)
    end

    A_bl=zeros(bus_number,branch_number)
    from_bus = 0
    to_bus = 0
    for i in 1:branch_number
        from_bus = ab_l[i][1]
        to_bus = ab_l[i][2]
        A_bl[(i-1)*bus_number+from_bus] = -1
        A_bl[(i-1)*bus_number+to_bus] = 1
    end 

    bus_p_max = zeros(1,bus_number)
        for i in 1:gen_number
            gen_index = string(i)
            bus_name = network["gen"][gen_index]["gen_bus"]
            bus_produce_max = network["gen"][gen_index]["pmax"]
            bus_p_max[bus_name] = bus_produce_max
        end

    bus_q_max = zeros(1,bus_number)
        for i in 1:gen_number
            gen_index = string(i)
            bus_name = network["gen"][gen_index]["gen_bus"]
            bus_q_max_value = network["gen"][gen_index]["qmax"]
            bus_q_max[bus_name] = bus_q_max_value
        end

    bus_p_min = zeros(1,bus_number)
        for i in 1:gen_number
            gen_index = string(i)
            bus_name = network["gen"][gen_index]["gen_bus"]
            bus_produce_min = network["gen"][gen_index]["pmin"]
            bus_p_min[bus_name] = bus_produce_min
        end

    bus_q_min = zeros(1,bus_number)
        for i in 1:gen_number
            gen_index = string(i)
            bus_name = network["gen"][gen_index]["gen_bus"]
            bus_q_min_value = network["gen"][gen_index]["qmin"]
            bus_q_min[bus_name] = bus_q_min_value
        end

    flow_bus_p_matrix = A_bl*flow_p
    flow_bus_q_matrix = A_bl*flow_q


    bus_v_min = []
    bus_v_max = []
    for i in 1:bus_number
        bus_index = string(i)
        push!(bus_v_min,network["bus"][bus_index]["vmin"])
        push!(bus_v_max,network["bus"][bus_index]["vmax"])
    end

    F_max = []
    for i in 1:branch_number
        branch_index = string(i)
        push!(F_max,network["branch"][branch_index]["rate_a"])
    end  

    B_b = zeros(1,bus_number)
    for i in 1:length(network["shunt"])
        i_string = string(i)
        bus_index = network["shunt"][i_string]["shunt_bus"]
        bus_susceptance =  network["shunt"][i_string]["bs"]
        B_b[bus_index] = bus_susceptance
    end

    bl = []
    gl = []
    for i in 1:branch_number
        branch_index = string(i)
        reactance = network["branch"][branch_index]["br_x"]
        resistance = network["branch"][branch_index]["br_r"]
        push!(bl,-reactance/(reactance^2+resistance^2))
        push!(gl,resistance/(reactance^2+resistance^2))
    end

# gen cost
   for i in 1:gen_number
       gen_index = string(i)
       bus_index = network["gen"][gen_index]["gen_bus"]
       @constraint(model_ac,gen_p[i] == bus_p[bus_index])
       @constraint(model_ac,gen_q[i] == bus_q[bus_index])
   end

   gen_cost_con=[]
   for i in 1:gen_number
       gen_index = string(i)
       push!(gen_cost_con,@constraint(model_ac,gen_cost[i] == network["gen"][gen_index]["cost"][1]*gen_p[i]^2+network["gen"][gen_index]["cost"][2]*gen_p[i]+network["gen"][gen_index]["cost"][3]))
   end
    
    
# Generation level constraints    
   for i in 1:bus_number
        if i in beta_g
            @constraint(model_ac, bus_q[i] <= bus_q_max[i])
            @constraint(model_ac, bus_q[i] >= bus_q_min[i])
        else
            @constraint(model_ac, bus_q[i] == 0)
        end
    end

    for i in 1:bus_number
        if i in beta_g
            @constraint(model_ac, bus_p[i] <= bus_p_max[i])
            @constraint(model_ac, bus_p[i] >= bus_p_min[i])
        else
            @constraint(model_ac, bus_p[i] == 0)
        end
    end

    con_p= []
    for i in 1:bus_number
        bus_index = string(i)
        push!(con_p,@constraint(model_ac, flow_bus_p_matrix[i]- bus_pd[i]== -bus_p[i] ))
    end
    con_q= []
    for i in 1:bus_number
        bus_index = string(i)
        push!(con_q,@constraint(model_ac, flow_bus_q_matrix[i]- bus_qd[i] + B_b[i]*(voltage_level[i])^2== -bus_q[i] ))
    end

    for i in 1:bus_number
        @constraint(model_ac, voltage_level[i] <= bus_v_max[i])
        @constraint(model_ac, voltage_level[i] >= bus_v_min[i])
    end

# Setting the Reference Bus (Bus 1) angles to 0
    @constraint(model_ac, voltage_phase[1] == 0)

# Thermal limits
    flow_con = []
    for i in 1:branch_number
        push!(flow_con,@constraint(model_ac, flow_p[i]^2 + flow_q[i]^2 <= F_max[i]^2))
    end 
    
# flow constrain
    @constraint(model_ac, flow_q[contingency] == 0)
    @constraint(model_ac, flow_p[contingency] == 0)    
    p_kvl = []
    q_kvl = []
    for i in 1:branch_number
        if i==contingency
        else
            
            branch_index = string(i)
            bus_fr_index = network["branch"][branch_index]["f_bus"]
            bus_to_index = network["branch"][branch_index]["t_bus"]    
            push!(p_kvl,@NLconstraint(model_ac,flow_p[i]== gl[i]*voltage_level[bus_fr_index]^2-voltage_level[bus_fr_index]*voltage_level[bus_to_index]*(gl[i]*cos(voltage_phase[bus_fr_index]-voltage_phase[bus_to_index])+bl[i]*sin(voltage_phase[bus_fr_index]-voltage_phase[bus_to_index]))))
            push!(q_kvl,@NLconstraint(model_ac,flow_q[i]== -bl[i]*voltage_level[bus_fr_index]^2-voltage_level[bus_fr_index]*voltage_level[bus_to_index]*(gl[i]*sin(voltage_phase[bus_fr_index]-voltage_phase[bus_to_index])-bl[i]*cos(voltage_phase[bus_fr_index]-voltage_phase[bus_to_index]))))
        end 
    end

# objective
    model = @objective(model_ac, Min,sum(gen_cost))
#@objective(model_dc, Min,sum(bus_p[1]+ bus_p[8]))
    optimize!(model_ac)
    
    status = termination_status(model_ac)
    if status == LOCALLY_SOLVED
        return(value.(sum(gen_cost)))
    else
        return("infeasible")
    end
    
end


run_ac_opf_withcontingency (generic function with 1 method)

In [127]:
infeasible_contingency = []
for i in 1:length(case_data["branch"])
    if run_ac_opf_withcontingency(case_data,i)=="infeasible"
        push!(infeasible_contingency,i)
    end
end

In [125]:
# this function is to compute the scopf problem with all feasible branch contingency
# the input is the dataset of the whole network 
# the output is solution of the problem if it has feasible, if it is not feasible the model will return "infeasible"
# this function is very similar to "run_ac_scopf_withcontingency"
function run_ac_opf_allcontingency(network)
    bus_number = length(network["bus"])
    gen_number = length(network["gen"])
    branch_number = length(network["branch"])
    contingency_number = length(network["branch"])
    
    model_ac = Model(with_optimizer(Ipopt.Optimizer))
    @variable(model_ac, flow_p[1:contingency_number,1:branch_number])
    @variable(model_ac, flow_q[1:contingency_number,1:branch_number])
    @variable(model_ac,voltage_phase[1:contingency_number,1:bus_number]) 
    @variable(model_ac,voltage_level[1:contingency_number,1:bus_number]) 
    @variable(model_ac, bus_p[1:bus_number])
    @variable(model_ac, bus_q[1:bus_number])
    @variable(model_ac, gen_p[1:gen_number])
    @variable(model_ac, gen_q[1:gen_number])
    @variable(model_ac, gen_cost[1:gen_number])

    beta_g = []
    for i in 1:gen_number
        gen_index = string(i)
        push!(beta_g,network["gen"][gen_index]["gen_bus"])
    end
    
   bus_gen = zeros(1,bus_number)
   for i in 1:gen_number
        index = beta_g[i]
        bus_gen[index] = 1
   end
         
   bus_pd = zeros(1,bus_number)
    for i in 1:length(network["load"])
        load_index = string(i)
        bus_name = network["load"][load_index]["load_bus"]
        bus_demand = network["load"][load_index]["pd"]
        bus_pd[bus_name] = bus_demand
    end

    bus_qd = zeros(1,bus_number)
    for i in 1:length(network["load"])
        load_index = string(i)
        bus_name = network["load"][load_index]["load_bus"]
        bus_demand = network["load"][load_index]["qd"]
        bus_qd[bus_name] = bus_demand
    end

    ab_l = []
    for i in 1:branch_number
        abl = []
        branch_index = string(i)
        push!(abl,network["branch"][branch_index]["f_bus"])
        push!(abl,network["branch"][branch_index]["t_bus"])
        push!(ab_l,abl)
    end

    A_bl=zeros(bus_number,branch_number)
    from_bus = 0
    to_bus = 0
    for i in 1:branch_number
        from_bus = ab_l[i][1]
        to_bus = ab_l[i][2]
        A_bl[(i-1)*bus_number+from_bus] = -1
        A_bl[(i-1)*bus_number+to_bus] = 1
    end 

    bus_p_max = zeros(1,bus_number)
        for i in 1:gen_number
            gen_index = string(i)
            bus_name = network["gen"][gen_index]["gen_bus"]
            bus_produce_max = network["gen"][gen_index]["pmax"]
            bus_p_max[bus_name] = bus_produce_max
        end

    bus_q_max = zeros(1,bus_number)
        for i in 1:gen_number
            gen_index = string(i)
            bus_name = network["gen"][gen_index]["gen_bus"]
            bus_q_max_value = network["gen"][gen_index]["qmax"]
            bus_q_max[bus_name] = bus_q_max_value
        end

    bus_p_min = zeros(1,bus_number)
        for i in 1:gen_number
            gen_index = string(i)
            bus_name = network["gen"][gen_index]["gen_bus"]
            bus_produce_min = network["gen"][gen_index]["pmin"]
            bus_p_min[bus_name] = bus_produce_min
        end

    bus_q_min = zeros(1,bus_number)
        for i in 1:gen_number
            gen_index = string(i)
            bus_name = network["gen"][gen_index]["gen_bus"]
            bus_q_min_value = network["gen"][gen_index]["qmin"]
            bus_q_min[bus_name] = bus_q_min_value
        end
    
    bus_v_min = []
    bus_v_max = []
    for i in 1:bus_number
        bus_index = string(i)
        push!(bus_v_min,network["bus"][bus_index]["vmin"])
        push!(bus_v_max,network["bus"][bus_index]["vmax"])
    end

    F_max = []
    for i in 1:branch_number
        branch_index = string(i)
        push!(F_max,network["branch"][branch_index]["rate_a"])
    end  

    B_b = zeros(1,bus_number)
    for i in 1:length(network["shunt"])
        i_string = string(i)
        bus_index = network["shunt"][i_string]["shunt_bus"]
        bus_susceptance =  network["shunt"][i_string]["bs"]
        B_b[bus_index] = bus_susceptance
    end

    bl = []
    gl = []
    for i in 1:branch_number
        branch_index = string(i)
        reactance = network["branch"][branch_index]["br_x"]
        resistance = network["branch"][branch_index]["br_r"]
        push!(bl,-reactance/(reactance^2+resistance^2))
        push!(gl,resistance/(reactance^2+resistance^2))
    end

# gen cost
   for i in 1:gen_number
       gen_index = string(i)
       bus_index = network["gen"][gen_index]["gen_bus"]
       @constraint(model_ac,gen_p[i] == bus_p[bus_index])
       @constraint(model_ac,gen_q[i] == bus_q[bus_index])
   end

   gen_cost_con=[]
   for i in 1:gen_number
       gen_index = string(i)
       push!(gen_cost_con,@constraint(model_ac,gen_cost[i] == network["gen"][gen_index]["cost"][1]*gen_p[i]^2+network["gen"][gen_index]["cost"][2]*gen_p[i]+network["gen"][gen_index]["cost"][3]))
   end
    
    
# Generation level constraints    
   for i in 1:bus_number
        if i in beta_g
            @constraint(model_ac, bus_q[i] <= bus_q_max[i])
            @constraint(model_ac, bus_q[i] >= bus_q_min[i])
        else
            @constraint(model_ac, bus_q[i] == 0)
        end
    end

    for i in 1:bus_number
        if i in beta_g
            @constraint(model_ac, bus_p[i] <= bus_p_max[i])
            @constraint(model_ac, bus_p[i] >= bus_p_min[i])
        else
            @constraint(model_ac, bus_p[i] == 0)
        end
    end

  # consraints under all contingency 
    for i in 1:contingency_number
        
        for j in 1:branch_number
            @constraint(model_ac, flow_p[i,j]^2+ flow_p[i,j]^2<= F_max[j]^2)
        end
    
    
        for j in 1:bus_number
            f_m_p = A_bl*flow_p[i,:]
            f_m_q = A_bl*flow_q[i,:]
            @constraint(model_ac, f_m_p[j] - bus_pd[j]== -bus_p[j] )
            @constraint(model_ac, f_m_q[j] - bus_qd[j]+ B_b[j]*(voltage_level[i,j])^2== -bus_q[j] )
        end
    
        for j in 1:bus_number
            @constraint(model_ac, voltage_level[i,j] <= bus_v_max[j])
            @constraint(model_ac, voltage_level[i,j] >= bus_v_min[j])
        end
        
    end   

  # consraints under all contingency 
    for i in 1:contingency_number

        if i in infeasible_contingency
            for j in 1:branch_number
                branch_index = string(j)
                bus_fr_index = network["branch"][branch_index]["f_bus"]
                bus_to_index = network["branch"][branch_index]["t_bus"]    
                @NLconstraint(model_ac,flow_p[i,j]== gl[j]*voltage_level[i,bus_fr_index]^2-voltage_level[i,bus_fr_index]*voltage_level[i,bus_to_index]*(gl[j]*cos(voltage_phase[i,bus_fr_index]-voltage_phase[i,bus_to_index])+bl[j]*sin(voltage_phase[i,bus_fr_index]-voltage_phase[i,bus_to_index])))
                @NLconstraint(model_ac,flow_q[i,j]== -bl[j]*voltage_level[i,bus_fr_index]^2-voltage_level[i,bus_fr_index]*voltage_level[i,bus_to_index]*(gl[j]*sin(voltage_phase[i,bus_fr_index]-voltage_phase[i,bus_to_index])-bl[j]*cos(voltage_phase[i,bus_fr_index]-voltage_phase[i,bus_to_index])))
            end           
        else   
            for j in 1:branch_number
                if j==i
                # the power flow in contingency branch is zero
                    @constraint(model_ac, flow_q[i,j] == 0)
                    @constraint(model_ac, flow_p[i,j] == 0)
                else
                    branch_index = string(j)
                    bus_fr_index = network["branch"][branch_index]["f_bus"]
                    bus_to_index = network["branch"][branch_index]["t_bus"]    
                    @NLconstraint(model_ac,flow_p[i,j]== gl[j]*voltage_level[i,bus_fr_index]^2-voltage_level[i,bus_fr_index]*voltage_level[i,bus_to_index]*(gl[j]*cos(voltage_phase[i,bus_fr_index]-voltage_phase[i,bus_to_index])+bl[j]*sin(voltage_phase[i,bus_fr_index]-voltage_phase[i,bus_to_index])))
                    @NLconstraint(model_ac,flow_q[i,j]== -bl[j]*voltage_level[i,bus_fr_index]^2-voltage_level[i,bus_fr_index]*voltage_level[i,bus_to_index]*(gl[j]*sin(voltage_phase[i,bus_fr_index]-voltage_phase[i,bus_to_index])-bl[j]*cos(voltage_phase[i,bus_fr_index]-voltage_phase[i,bus_to_index])))
                end 
            end
        end
    
    end   

    # objective
    model = @objective(model_ac, Min,sum(gen_cost))
    #@objective(model_dc, Min,sum(bus_p[1]+ bus_p[8]))
    optimize!(model_ac)

    status = termination_status(model_ac)
    if status == LOCALLY_SOLVED
        return(value.(gen_p),value.(gen_q))
    else
        return("infeasible")
    end
    
end


run_ac_opf_allcontingency (generic function with 1 method)

In [109]:
# this function is to find all the violation branch under all contingency
# the input is the dataset of the whole network
# the output is the number of violation contingencies, the index of these branches, their violation value and the corresopnding lagrange multiplier
# this can be used to compute all the sub problems for a grid
function benders_subproblem(network)
    contingency_violation = []
    lagrangian_multiplier_set_p = []
    lagrangian_multiplier_set_v = []
    violate_branch = []
    violation_l = []
    
    for contingency in 1:length(network["branch"])
        bus_number = length(network["bus"])
        gen_number = length(network["gen"])
        branch_number = length(network["branch"])

        model_ac_slack = Model(with_optimizer(Ipopt.Optimizer,print_level=0))
        @variable(model_ac_slack, slack1[1:branch_number],lower_bound=0)
        @variable(model_ac_slack, slack2[1:branch_number],lower_bound=0)
        @variable(model_ac_slack, slack3[1:bus_number],lower_bound=0)
        @variable(model_ac_slack, slack4[1:bus_number],lower_bound=0)
        @variable(model_ac_slack, slack5[1:bus_number],lower_bound=0)
        @variable(model_ac_slack, slack6[1:bus_number],lower_bound=0)
        @variable(model_ac_slack, flow_p[1:branch_number])
        @variable(model_ac_slack, flow_q[1:branch_number])
        @variable(model_ac_slack,voltage_phase[1:bus_number]) 
        @variable(model_ac_slack,voltage_level[1:bus_number])  
        @variable(model_ac_slack, bus_q[1:bus_number])
        
        
        beta_g = []
        for i in 1:gen_number
            gen_index = string(i)
            push!(beta_g,network["gen"][gen_index]["gen_bus"])
        end
    
       bus_gen = zeros(1,bus_number)
        for i in 1:gen_number
            index = beta_g[i]
            bus_gen[index] = 1
        end
        
        bus_p0 = zeros(1,bus_number)
        for i in 1:gen_number
            gen_index = string(i)
            bus_name = network["gen"][gen_index]["gen_bus"]
            bus_produce = network["gen"][gen_index]["pg"]
            bus_p0[bus_name] = bus_produce
        end   

        bus_pd = zeros(1,bus_number)
        for i in 1:length(network["load"])
            load_index = string(i)
            bus_name = network["load"][load_index]["load_bus"]
            bus_demand = network["load"][load_index]["pd"]
            bus_pd[bus_name] = bus_demand
        end

        bus_qd = zeros(1,bus_number)
        for i in 1:length(network["load"])
            load_index = string(i)
            bus_name = network["load"][load_index]["load_bus"]
            bus_demand = network["load"][load_index]["qd"]
            bus_qd[bus_name] = bus_demand
        end

        ab_l = []
        for i in 1:branch_number
            abl = []
            branch_index = string(i)
            push!(abl,network["branch"][branch_index]["f_bus"])
            push!(abl,network["branch"][branch_index]["t_bus"])
            push!(ab_l,abl)
        end

        A_bl=zeros(bus_number,branch_number)
        from_bus = 0
        to_bus = 0
        for i in 1:branch_number
            from_bus = ab_l[i][1]
            to_bus = ab_l[i][2]
            A_bl[(i-1)*bus_number+from_bus] = -1
            A_bl[(i-1)*bus_number+to_bus] = 1
        end
        
        bus_v_min = []
        bus_v_max = []
        for i in 1:bus_number
            bus_index = string(i)
            push!(bus_v_min,network["bus"][bus_index]["vmin"])
            push!(bus_v_max,network["bus"][bus_index]["vmax"])
        end

        F_max = []
        for i in 1:branch_number
            branch_index = string(i)
            push!(F_max,network["branch"][branch_index]["rate_a"])
        end  

        B_b = zeros(1,bus_number)
        for i in 1:length(network["shunt"])
            i_string = string(i)
            bus_index = network["shunt"][i_string]["shunt_bus"]
            bus_susceptance =  network["shunt"][i_string]["bs"]
            B_b[bus_index] = bus_susceptance
        end

        bl = []
        gl = []
        for i in 1:branch_number
            branch_index = string(i)
            reactance = network["branch"][branch_index]["br_x"]
            resistance = network["branch"][branch_index]["br_r"]
            push!(bl,-reactance/(reactance^2+resistance^2))
            push!(gl,resistance/(reactance^2+resistance^2))
        end
        
        flow_bus_p_matrix = A_bl*flow_p
        flow_bus_q_matrix = A_bl*flow_q
        
        
        bus_q_max = zeros(1,bus_number)
        for i in 1:gen_number
            gen_index = string(i)
            bus_name = network["gen"][gen_index]["gen_bus"]
            bus_q_max_value = network["gen"][gen_index]["qmax"]
            bus_q_max[bus_name] = bus_q_max_value
        end

        bus_q_min = zeros(1,bus_number)
        for i in 1:gen_number
            gen_index = string(i)
            bus_name = network["gen"][gen_index]["gen_bus"]
            bus_q_min_value = network["gen"][gen_index]["qmin"]
            bus_q_min[bus_name] = bus_q_min_value
        end

        for i in 1:bus_number
            @constraint(model_ac_slack, bus_q[i] <= bus_q_max[i]+ slack5[i])
            @constraint(model_ac_slack, bus_q[i] >= bus_q_min[i]- slack6[i])
        end

    

        con_v = []
        for i in 1:bus_number
            bus_index = string(i)
            if i in beta_g
                push!(con_v,@constraint(model_ac_slack, network["bus"][bus_index]["vm"] == voltage_level[i]))
            end
        end
      
        
        con_p= []
        for i in 1:bus_number
            bus_index = string(i)
            push!(con_p,@constraint(model_ac_slack, flow_bus_p_matrix[i]- bus_pd[i]+ slack3[i] - slack4[i]== -bus_p0[i] ))
        end
        con_q= []
        for i in 1:bus_number
            bus_index = string(i)
            push!(con_q,@constraint(model_ac_slack, flow_bus_q_matrix[i]- bus_qd[i] + B_b[i]*(voltage_level[i])^2  == -bus_q[i] ))
        end

        for i in 1:bus_number
            @constraint(model_ac_slack, voltage_level[i] <= bus_v_max[i])
            @constraint(model_ac_slack, voltage_level[i] >= bus_v_min[i])
        end

# Setting the Reference Bus (Bus 1) angles to 0
        @constraint(model_ac_slack, voltage_phase[1] == 0)

# Thermal limits
        flow_con = []
        for i in 1:branch_number
            push!(flow_con,@NLconstraint(model_ac_slack, flow_p[i]^2 + flow_q[i]^2<= (F_max[i]*1.5 + slack1[i])^2))
        end 

        
# flow constrain
        @constraint(model_ac_slack, flow_p[contingency] == 0)
        @constraint(model_ac_slack, flow_q[contingency] == 0)
        p_kvl = []
        q_kvl = []
        for i in 1:(contingency-1)
            branch_index = string(i)
            bus_fr_index = network["branch"][branch_index]["f_bus"]
            bus_to_index = network["branch"][branch_index]["t_bus"]    
            push!(p_kvl,@NLconstraint(model_ac_slack,flow_p[i]== gl[i]*voltage_level[bus_fr_index]^2-voltage_level[bus_fr_index]*voltage_level[bus_to_index]*(gl[i]*cos(voltage_phase[bus_fr_index]-voltage_phase[bus_to_index])+bl[i]*sin(voltage_phase[bus_fr_index]-voltage_phase[bus_to_index]))))
            push!(q_kvl,@NLconstraint(model_ac_slack,flow_q[i]== -bl[i]*voltage_level[bus_fr_index]^2-voltage_level[bus_fr_index]*voltage_level[bus_to_index]*(gl[i]*sin(voltage_phase[bus_fr_index]-voltage_phase[bus_to_index])-bl[i]*cos(voltage_phase[bus_fr_index]-voltage_phase[bus_to_index]))))
        end 
        
        for i in (contingency+1):branch_number
            branch_index = string(i)
            bus_fr_index = network["branch"][branch_index]["f_bus"]
            bus_to_index = network["branch"][branch_index]["t_bus"]    
            push!(p_kvl,@NLconstraint(model_ac_slack,flow_p[i]== gl[i]*voltage_level[bus_fr_index]^2-voltage_level[bus_fr_index]*voltage_level[bus_to_index]*(gl[i]*cos(voltage_phase[bus_fr_index]-voltage_phase[bus_to_index])+bl[i]*sin(voltage_phase[bus_fr_index]-voltage_phase[bus_to_index]))))
            push!(q_kvl,@NLconstraint(model_ac_slack,flow_q[i]== -bl[i]*voltage_level[bus_fr_index]^2-voltage_level[bus_fr_index]*voltage_level[bus_to_index]*(gl[i]*sin(voltage_phase[bus_fr_index]-voltage_phase[bus_to_index])-bl[i]*cos(voltage_phase[bus_fr_index]-voltage_phase[bus_to_index]))))
        end 
# objective
        violation = @objective(model_ac_slack, Min,sum(sum(sum(slack1[i] for i in [1:branch_number]))+sum(sum(slack4[i] for i in [1:bus_number])+sum(slack3[i] for i in [1:bus_number])+sum(slack5[i] for i in [1:bus_number])+sum(slack6[i] for i in [1:bus_number]))))
        flow_violation = sum(sum(slack1[i] for i in [1:branch_number]))
        optimize!(model_ac_slack)    
    
        # if value.(flow_violation)>10e-8 && (value.(violation) -value.(flow_violation))<10e-8
        
        if value.(flow_violation)>10e-8
             push!(lagrangian_multiplier_set_p,-dual.(con_p))
             push!(lagrangian_multiplier_set_v,dual.(con_v))
             push!(violate_branch,contingency)
             push!(contingency_violation,value.(violation))
             #push!(contingency_violation,value.(flow_violation))
             push!(violation_l,value.(violation))
        end         
    end
    num = length(violate_branch)
    
    return (num,violate_branch,contingency_violation,lagrangian_multiplier_set_p, lagrangian_multiplier_set_v)

end


benders_subproblem (generic function with 1 method)

In [110]:
# this function is to find all the power flow violation branch for a specific contingency
# the input is the dataset of the whole network and a contingency
# the output is to tell whether the critical contingency is power flow violation contingency or not
# this can only be used to check one contingency for each time
# this function is almost the same with the above one
function benders_subproblem_contingency(network,contingency::Int64)
    contingency_violation = []
    lagrangian_multiplier_set_p = []
    lagrangian_multiplier_set_q = []
    violate_branch = []
    
    

    bus_number = length(network["bus"])
    gen_number = length(network["gen"])
    branch_number = length(network["branch"])

    model_ac_slack = Model(with_optimizer(Ipopt.Optimizer,print_level=0))
    @variable(model_ac_slack, slack1[1:branch_number],lower_bound=0)
    @variable(model_ac_slack, slack2[1:branch_number],lower_bound=0)
    @variable(model_ac_slack, slack3[1:bus_number],lower_bound=0)
    @variable(model_ac_slack, slack4[1:bus_number],lower_bound=0)
    @variable(model_ac_slack, slack5[1:bus_number],lower_bound=0)
    @variable(model_ac_slack, slack6[1:bus_number],lower_bound=0)
    @variable(model_ac_slack, flow_p[1:branch_number])
    @variable(model_ac_slack, flow_q[1:branch_number])
    @variable(model_ac_slack,voltage_phase[1:bus_number]) 
    @variable(model_ac_slack,voltage_level[1:bus_number])  
    @variable(model_ac_slack, bus_q[1:bus_number])
        
    beta_g = []
    for i in 1:gen_number
        gen_index = string(i)
        push!(beta_g,network["gen"][gen_index]["gen_bus"])
    end
    
    bus_gen = zeros(1,bus_number)
    for i in 1:gen_number
        index = beta_g[i]
        bus_gen[index] = 1
    end
        
    bus_p0 = zeros(1,bus_number)
    for i in 1:gen_number
        gen_index = string(i)
        bus_name = network["gen"][gen_index]["gen_bus"]
        bus_produce = network["gen"][gen_index]["pg"]
        bus_p0[bus_name] = bus_produce
    end 
    
    bus_pd = zeros(1,bus_number)
    for i in 1:length(network["load"])
        load_index = string(i)
        bus_name = network["load"][load_index]["load_bus"]
        bus_demand = network["load"][load_index]["pd"]
        bus_pd[bus_name] = bus_demand
    end

    bus_qd = zeros(1,bus_number)
    for i in 1:length(network["load"])
        load_index = string(i)
        bus_name = network["load"][load_index]["load_bus"]
        bus_demand = network["load"][load_index]["qd"]
        bus_qd[bus_name] = bus_demand
    end

    ab_l = []
    for i in 1:branch_number
        abl = []
        branch_index = string(i)
        push!(abl,network["branch"][branch_index]["f_bus"])
        push!(abl,network["branch"][branch_index]["t_bus"])
        push!(ab_l,abl)
    end

    A_bl=zeros(bus_number,branch_number)
    from_bus = 0
    to_bus = 0
    for i in 1:branch_number
        from_bus = ab_l[i][1]
        to_bus = ab_l[i][2]
        A_bl[(i-1)*bus_number+from_bus] = -1
        A_bl[(i-1)*bus_number+to_bus] = 1
    end
        
    bus_v_min = []
    bus_v_max = []
    for i in 1:bus_number
        bus_index = string(i)
        push!(bus_v_min,network["bus"][bus_index]["vmin"])
        push!(bus_v_max,network["bus"][bus_index]["vmax"])
    end

    F_max = []
    for i in 1:branch_number
        branch_index = string(i)
        push!(F_max,network["branch"][branch_index]["rate_a"])
    end  

    B_b = zeros(1,bus_number)
    for i in 1:length(network["shunt"])
        i_string = string(i)
        bus_index = network["shunt"][i_string]["shunt_bus"]
        bus_susceptance =  network["shunt"][i_string]["bs"]
        B_b[bus_index] = bus_susceptance
    end

    bl = []
    gl = []
    for i in 1:branch_number
        branch_index = string(i)
        reactance = network["branch"][branch_index]["br_x"]
        resistance = network["branch"][branch_index]["br_r"]
        push!(bl,-reactance/(reactance^2+resistance^2))
        push!(gl,resistance/(reactance^2+resistance^2))
    end
        
    flow_bus_p_matrix = A_bl*flow_p
    flow_bus_q_matrix = A_bl*flow_q
    
    bus_q_max = zeros(1,bus_number)
    for i in 1:gen_number
        gen_index = string(i)
        bus_name = network["gen"][gen_index]["gen_bus"]
        bus_q_max_value = network["gen"][gen_index]["qmax"]
        bus_q_max[bus_name] = bus_q_max_value
    end

    bus_q_min = zeros(1,bus_number)
    for i in 1:gen_number
        gen_index = string(i)
        bus_name = network["gen"][gen_index]["gen_bus"]
        bus_q_min_value = network["gen"][gen_index]["qmin"]
        bus_q_min[bus_name] = bus_q_min_value
    end
    

    for i in 1:bus_number
        bus_index = string(i)
        if i in beta_g
            @constraint(model_ac_slack,voltage_level[i] == network["bus"][bus_index]["vm"])
        end
    end
        
        
    con_p= []
    for i in 1:bus_number
        bus_index = string(i)
        push!(con_p,@constraint(model_ac_slack, flow_bus_p_matrix[i]- bus_pd[i]+ slack3[i] - slack4[i]== -bus_p0[i] ))
    end
    con_q= []
    for i in 1:bus_number
        bus_index = string(i)
        push!(con_q,@constraint(model_ac_slack, flow_bus_q_matrix[i]- bus_qd[i] + B_b[i]*(voltage_level[i])^2== -bus_q[i] ))
    end

    for i in 1:bus_number
        @constraint(model_ac_slack, voltage_level[i] <= bus_v_max[i])
        @constraint(model_ac_slack, voltage_level[i] >= bus_v_min[i])
    end
    
    for i in 1:bus_number
        @constraint(model_ac_slack, bus_q[i] <= bus_q_max[i]+ slack5[i] )
        @constraint(model_ac_slack, bus_q[i] >= bus_q_min[i] -slack6[i] )
    end

    
    # Setting the Reference Bus (Bus 1) angles to 0
    @constraint(model_ac_slack, voltage_phase[1] == 0)

    # Thermal limits
        flow_con = []
        for i in 1:branch_number
            push!(flow_con,@NLconstraint(model_ac_slack, flow_p[i]^2 + flow_q[i]^2 <= (F_max[i]*1.5 +slack1[i])^2 ))
        end 


# flow constrain
    @constraint(model_ac_slack, flow_p[contingency] == 0)
    @constraint(model_ac_slack, flow_q[contingency] == 0)
    p_kvl = []
    q_kvl = []
    for i in 1:(contingency-1)
        branch_index = string(i)
        bus_fr_index = network["branch"][branch_index]["f_bus"]
        bus_to_index = network["branch"][branch_index]["t_bus"]    
        push!(p_kvl,@NLconstraint(model_ac_slack,flow_p[i]== gl[i]*voltage_level[bus_fr_index]^2-voltage_level[bus_fr_index]*voltage_level[bus_to_index]*(gl[i]*cos(voltage_phase[bus_fr_index]-voltage_phase[bus_to_index])+bl[i]*sin(voltage_phase[bus_fr_index]-voltage_phase[bus_to_index]))))
        push!(q_kvl,@NLconstraint(model_ac_slack,flow_q[i]== -bl[i]*voltage_level[bus_fr_index]^2-voltage_level[bus_fr_index]*voltage_level[bus_to_index]*(gl[i]*sin(voltage_phase[bus_fr_index]-voltage_phase[bus_to_index])-bl[i]*cos(voltage_phase[bus_fr_index]-voltage_phase[bus_to_index]))))
    end 
        
    for i in (contingency+1):branch_number
        branch_index = string(i)
        bus_fr_index = network["branch"][branch_index]["f_bus"]
        bus_to_index = network["branch"][branch_index]["t_bus"]    
        push!(p_kvl,@NLconstraint(model_ac_slack,flow_p[i]== gl[i]*voltage_level[bus_fr_index]^2-voltage_level[bus_fr_index]*voltage_level[bus_to_index]*(gl[i]*cos(voltage_phase[bus_fr_index]-voltage_phase[bus_to_index])+bl[i]*sin(voltage_phase[bus_fr_index]-voltage_phase[bus_to_index]))))
        push!(q_kvl,@NLconstraint(model_ac_slack,flow_q[i]== -bl[i]*voltage_level[bus_fr_index]^2-voltage_level[bus_fr_index]*voltage_level[bus_to_index]*(gl[i]*sin(voltage_phase[bus_fr_index]-voltage_phase[bus_to_index])-bl[i]*cos(voltage_phase[bus_fr_index]-voltage_phase[bus_to_index]))))
    end 
# objective
    violation = @objective(model_ac_slack, Min,sum(sum(sum(slack1[i] for i in [1:branch_number]))+sum(sum(slack4[i] for i in [1:bus_number])+sum(slack4[i] for i in [1:bus_number])+sum(slack5[i] for i in [1:bus_number])+sum(slack6[i] for i in [1:bus_number]))))

    optimize!(model_ac_slack)    
    return (sum(value.(slack1)))
   
end


benders_subproblem_contingency (generic function with 1 method)

In [131]:
# this function is add all benders cuts to the master problem  
# the input is the dataset of the whole network , the critical contingency list, violation value list , lagrangian multiplier list, value of given bus generation and voltage level
# the output is solution of basic dcopf model with a benders cut
# this function can add all cuts 
# this is the master problem for Bender method
function run_ac_opf_all_benderscut(network,lagrangian_multiplier_set_p::Array,lagrangian_multiplier_set_v::Array,contingency_violation::Array,effective_violate_branch_list::Array)
    bus_number = length(network["bus"])
    gen_number = length(network["gen"])
    branch_number = length(network["branch"])

    model_ac = Model(with_optimizer(Ipopt.Optimizer,print_level = 0))
    @variable(model_ac, flow_p[1:branch_number])
    @variable(model_ac, flow_q[1:branch_number])
    @variable(model_ac,voltage_phase[1:bus_number]) 
    @variable(model_ac,voltage_level[1:bus_number]) 
    @variable(model_ac, bus_p[1:bus_number])
    @variable(model_ac, bus_q[1:bus_number])
    @variable(model_ac, gen_p[1:gen_number])
    @variable(model_ac, gen_q[1:gen_number])
    @variable(model_ac, gen_cost[1:gen_number])
    @variable(model_ac, bus_add_cut1[1:bus_number])
    @variable(model_ac, bus_add_cut2[1:bus_number])
    @variable(model_ac,voltage_level_gen[1:bus_number]) 

    beta_g = []
    for i in 1:gen_number
        gen_index = string(i)
        push!(beta_g,network["gen"][gen_index]["gen_bus"])
    end
    
   bus_gen = zeros(1,bus_number)
   for i in 1:gen_number
        index = beta_g[i]
        bus_gen[index] = 1
   end
         
   bus_pd = zeros(1,bus_number)
    for i in 1:length(network["load"])
        load_index = string(i)
        bus_name = network["load"][load_index]["load_bus"]
        bus_demand = network["load"][load_index]["pd"]
        bus_pd[bus_name] = bus_demand
    end

    bus_qd = zeros(1,bus_number)
    for i in 1:length(network["load"])
        load_index = string(i)
        bus_name = network["load"][load_index]["load_bus"]
        bus_demand = network["load"][load_index]["qd"]
        bus_qd[bus_name] = bus_demand
    end
    
    bus_p0 = zeros(1,bus_number)
    for i in 1:gen_number
        gen_index = string(i)
        bus_name = network["gen"][gen_index]["gen_bus"]
        bus_produce = network["gen"][gen_index]["pg"]
        bus_p0[bus_name] = bus_produce
    end



    ab_l = []
    for i in 1:branch_number
        abl = []
        branch_index = string(i)
        push!(abl,network["branch"][branch_index]["f_bus"])
        push!(abl,network["branch"][branch_index]["t_bus"])
        push!(ab_l,abl)
    end

    A_bl=zeros(bus_number,branch_number)
    from_bus = 0
    to_bus = 0
    for i in 1:branch_number
        from_bus = ab_l[i][1]
        to_bus = ab_l[i][2]
        A_bl[(i-1)*bus_number+from_bus] = -1
        A_bl[(i-1)*bus_number+to_bus] = 1
    end 

    bus_p_max = zeros(1,bus_number)
        for i in 1:gen_number
            gen_index = string(i)
            bus_name = network["gen"][gen_index]["gen_bus"]
            bus_produce_max = network["gen"][gen_index]["pmax"]
            bus_p_max[bus_name] = bus_produce_max
        end

    bus_q_max = zeros(1,bus_number)
        for i in 1:gen_number
            gen_index = string(i)
            bus_name = network["gen"][gen_index]["gen_bus"]
            bus_q_max_value = network["gen"][gen_index]["qmax"]
            bus_q_max[bus_name] = bus_q_max_value
        end

    bus_p_min = zeros(1,bus_number)
        for i in 1:gen_number
            gen_index = string(i)
            bus_name = network["gen"][gen_index]["gen_bus"]
            bus_produce_min = network["gen"][gen_index]["pmin"]
            bus_p_min[bus_name] = bus_produce_min
        end

    bus_q_min = zeros(1,bus_number)
        for i in 1:gen_number
            gen_index = string(i)
            bus_name = network["gen"][gen_index]["gen_bus"]
            bus_q_min_value = network["gen"][gen_index]["qmin"]
            bus_q_min[bus_name] = bus_q_min_value
        end

    flow_bus_p_matrix = A_bl*flow_p
    flow_bus_q_matrix = A_bl*flow_q



    bus_v_min = []
    bus_v_max = []
    for i in 1:bus_number
        bus_index = string(i)
        push!(bus_v_min,network["bus"][bus_index]["vmin"])
        push!(bus_v_max,network["bus"][bus_index]["vmax"])
    end

    F_max = []
    for i in 1:branch_number
        branch_index = string(i)
        push!(F_max,network["branch"][branch_index]["rate_a"])
    end  

    B_b = zeros(1,bus_number)
    for i in 1:length(network["shunt"])
        i_string = string(i)
        bus_index = network["shunt"][i_string]["shunt_bus"]
        bus_susceptance =  network["shunt"][i_string]["bs"]
        B_b[bus_index] = bus_susceptance
    end

    bl = []
    gl = []
    for i in 1:branch_number
        branch_index = string(i)
        reactance = network["branch"][branch_index]["br_x"]
        resistance = network["branch"][branch_index]["br_r"]
        push!(bl,-reactance/(reactance^2+resistance^2))
        push!(gl,resistance/(reactance^2+resistance^2))
    end
    
    voltage_level0 = []
    for i in 1:bus_number
        if i in beta_g
            push!(voltage_level0,network["bus"][string(i)]["vm"])
        end
    end
    
    for i in 1:gen_number
       gen_index = string(i)
       bus_index = network["gen"][gen_index]["gen_bus"]
       @constraint(model_ac,voltage_level_gen[i] == voltage_level[bus_index])
    end
        

# gen cost
   for i in 1:gen_number
       gen_index = string(i)
       bus_index = network["gen"][gen_index]["gen_bus"]
       @constraint(model_ac,gen_p[i] == bus_p[bus_index])
       @constraint(model_ac,gen_q[i] == bus_q[bus_index])
   end

   gen_cost_con=[]
   for i in 1:gen_number
       gen_index = string(i)
       push!(gen_cost_con,@constraint(model_ac,gen_cost[i] == network["gen"][gen_index]["cost"][1]*gen_p[i]^2+network["gen"][gen_index]["cost"][2]*gen_p[i]+network["gen"][gen_index]["cost"][3]))
   end
    
    
#bender cut
    vio_num = length(effective_violate_branch_list)
    for j in 1:vio_num
        index= effective_violate_branch_list[j]
        @constraint(model_ac, sum((bus_p[i]-bus_p0[i])*lagrangian_multiplier_set_p[index][i] for i in 1:bus_number)+sum((voltage_level_gen[k]-voltage_level0[k])*lagrangian_multiplier_set_v[index][k] for k in 1:gen_number)+contingency_violation_list[index]<=0)
    end
    
    
# Generation level constraints    
   for i in 1:bus_number
        if i in beta_g
            @constraint(model_ac, bus_q[i] <= bus_q_max[i])
            @constraint(model_ac, bus_q[i] >= bus_q_min[i])
        else
            @constraint(model_ac, bus_q[i] == 0)
        end
    end

    for i in 1:bus_number
        if i in beta_g
            @constraint(model_ac, bus_p[i] <= bus_p_max[i])
            @constraint(model_ac, bus_p[i] >= bus_p_min[i])
        else
            @constraint(model_ac, bus_p[i] == 0)
        end
    end

    con_p= []
    for i in 1:bus_number
        bus_index = string(i)
        push!(con_p,@constraint(model_ac, flow_bus_p_matrix[i]- bus_pd[i]== -bus_p[i] ))
    end
    con_q= []
    for i in 1:bus_number
        bus_index = string(i)
        push!(con_q,@constraint(model_ac, flow_bus_q_matrix[i]- bus_qd[i] + B_b[i]*(voltage_level[i])^2== -bus_q[i] ))
    end

    for i in 1:bus_number
        @constraint(model_ac, voltage_level[i] <= bus_v_max[i])
        @constraint(model_ac, voltage_level[i] >= bus_v_min[i])
    end

# Setting the Reference Bus (Bus 1) angles to 0
    @constraint(model_ac, voltage_phase[1] == 0)

# Thermal limits
    flow_con = []
    for i in 1:branch_number
        push!(flow_con,@constraint(model_ac, flow_p[i]^2 + flow_q[i]^2 <= F_max[i]^2))
    end 

# flow constrain
    p_kvl = []
    q_kvl = []
    for i in 1:branch_number
        branch_index = string(i)
        bus_fr_index = network["branch"][branch_index]["f_bus"]
        bus_to_index = network["branch"][branch_index]["t_bus"]    
        push!(p_kvl,@NLconstraint(model_ac,flow_p[i]== gl[i]*voltage_level[bus_fr_index]^2-voltage_level[bus_fr_index]*voltage_level[bus_to_index]*(gl[i]*cos(voltage_phase[bus_fr_index]-voltage_phase[bus_to_index])+bl[i]*sin(voltage_phase[bus_fr_index]-voltage_phase[bus_to_index]))))
        push!(q_kvl,@NLconstraint(model_ac,flow_q[i]== -bl[i]*voltage_level[bus_fr_index]^2-voltage_level[bus_fr_index]*voltage_level[bus_to_index]*(gl[i]*sin(voltage_phase[bus_fr_index]-voltage_phase[bus_to_index])-bl[i]*cos(voltage_phase[bus_fr_index]-voltage_phase[bus_to_index]))))
    end 

# objective
    model = @objective(model_ac, Min,sum(gen_cost))

    optimize!(model_ac)
    # return (value.(gen_p),value.(voltage_level))
    status = termination_status(model_ac)
    if status == LOCALLY_SOLVED
        return(value.(gen_p),value.(voltage_level))
    else
        return("infeasible")
    end
end


run_ac_opf_all_benderscut (generic function with 1 method)

In [154]:
# set the initial value of control variables 
network_base_con = deepcopy(case_data)
for i in 1:length(network_base_con["gen"])
    network_base_con["gen"][string(i)]["pg"] = basic_gen_p[i]  
end

for i in 1:length(network_base_con["bus"])
    network_base_con["bus"][string(i)]["vm"] = basic_voltage_level[i]
end

In [155]:
# first iteration to find violation continegncy
network_con = deepcopy(network_base_con)
violation_details = benders_subproblem(network_con)

vd = collect(violation_details)
violation_number = vd[1]
violate_branch_list = vd[2]
contingency_violation_list = vd[3]
lagrangian_multiplier_set_p = vd[4]
lagrangian_multiplier_set_v = vd[5]

if violation_number == 0
    info(LOGGER, "we find the optimal solution to satisfy all branch contingencies")
else
    info(LOGGER, "we DO NOT the optimal solution to satisfy all branch contingencies")
    println("viloation number:$(violation_number)")
    println("violate branch list:$(violate_branch_list)")
    println("contingency violation value list:$(contingency_violation_list)")
end

[info | GOC]: we DO NOT the optimal solution to satisfy all branch contingencies
viloation number:6
violate branch list:Any[7, 9, 16, 19, 32, 36]
contingency violation value list:Any[0.0993136311181749, 0.16592650329431022, 0.38838064149503626, 0.005724416049003831, 0.002348858875712079, 0.2523415104989777]


In [156]:
# filter contingency
network = deepcopy(network_base_con)
delete_list = []
for i in 1:violation_number 
    a = [i]
    new = run_ac_opf_all_benderscut(network,lagrangian_multiplier_set_p,lagrangian_multiplier_set_v,contingency_violation_list,a)
    if new == "infeasible"
        push!(delete_list,i)
    end
end

violate_branch_list = deleteat!(violate_branch_list, delete_list)
if length(violate_branch_list) == 0
    info(LOGGER, "we find the optimal solution to satisfy all branch contingencies")
else
    info(LOGGER, "we DO NOT the optimal solution to satisfy all branch contingencies")
    println("violate branch list:$(violate_branch_list)")
end

[info | GOC]: we find the optimal solution to satisfy all branch contingencies
